In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Unified_Text.csv")
df.head()

,Unnamed: 0,log_type,source_ip,component,level,status,event_id,content,Year,Month,Day,Hour,Minute,log_text
0,0,Access,1.202.218.8,NaN,NaN,404,NaN,GET /robots.txt HTTP/1.0,2012.0,6.0,20.0,19.0,5.0,"Log Type: Access, Source IP: 1.202.218.8, Leve..."
1,1,Access,208.115.113.91,NaN,NaN,200,NaN,GET /logs/?C=M;O=D HTTP/1.1,2012.0,6.0,20.0,19.0,20.0,"Log Type: Access, Source IP: 208.115.113.91, L..."
2,2,Access,123.125.71.20,NaN,NaN,200,NaN,GET / HTTP/1.1,2012.0,6.0,20.0,19.0,30.0,"Log Type: Access, Source IP: 123.125.71.20, Le..."
3,3,Access,220.181.108.101,NaN,NaN,200,NaN,GET / HTTP/1.1,2012.0,6.0,20.0,19.0,31.0,"Log Type: Access, Source IP: 220.181.108.101, ..."
4,4,Access,123.125.68.79,NaN,NaN,200,NaN,GET / HTTP/1.1,2012.0,6.0,20.0,19.0,53.0,"Log Type: Access, Source IP: 123.125.68.79, Le..."


In [3]:
df.fillna('',inplace=True)

<ipython-input-3-3094999256>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('',inplace=True)


In [4]:
df.head()

,Unnamed: 0,log_type,source_ip,component,level,status,event_id,content,Year,Month,Day,Hour,Minute,log_text
0,0,Access,1.202.218.8,,,404,,GET /robots.txt HTTP/1.0,2012.0,6.0,20.0,19.0,5.0,"Log Type: Access, Source IP: 1.202.218.8, Leve..."
1,1,Access,208.115.113.91,,,200,,GET /logs/?C=M;O=D HTTP/1.1,2012.0,6.0,20.0,19.0,20.0,"Log Type: Access, Source IP: 208.115.113.91, L..."
2,2,Access,123.125.71.20,,,200,,GET / HTTP/1.1,2012.0,6.0,20.0,19.0,30.0,"Log Type: Access, Source IP: 123.125.71.20, Le..."
3,3,Access,220.181.108.101,,,200,,GET / HTTP/1.1,2012.0,6.0,20.0,19.0,31.0,"Log Type: Access, Source IP: 220.181.108.101, ..."
4,4,Access,123.125.68.79,,,200,,GET / HTTP/1.1,2012.0,6.0,20.0,19.0,53.0,"Log Type: Access, Source IP: 123.125.68.79, Le..."


# TF-IDF from Scratch

**Tokenize Each Log Message**

In [5]:
def tokenize(text):
  return text.lower().split()


df["tokens"] = df["log_text"].apply(tokenize)

**Compute Term Frequency (TF)**

In [6]:
from collections import Counter

def compute_tf(tokens):
    tf = {}
    word_counts = Counter(tokens)
    total_terms = len(tokens)
    for word, count in word_counts.items():
        tf[word] = count / total_terms
    return tf

df["tf"] = df["tokens"].apply(compute_tf)


**Compute Inverse Document Frequency (IDF**

In [7]:
import math
def compute_idf(documents):
    N = len(documents)
    idf = {}
    all_words = set(word for doc in documents for word in doc)

    for word in all_words:
        df_count = sum(1 for doc in documents if word in doc)
        idf[word] = math.log(N / (1 + df_count))
    return idf

idf = compute_idf(df["tokens"])


**Compute TF-IDF**

In [8]:
def compute_tfidf(tf, idf):
    tfidf = {}
    for word, tf_val in tf.items():
        tfidf[word] = tf_val * idf.get(word, 0.0)
    return tfidf

df["tfidf"] = df["tf"].apply(lambda tf: compute_tfidf(tf, idf))


**View Final DataFrame**

In [9]:
df["tfidf"]


,tfidf
0,"{'log': -9.263805386224244e-07, 'type:': -9.26..."
1,"{'log': -9.263805386224244e-07, 'type:': -9.26..."
2,"{'log': -9.263805386224244e-07, 'type:': -9.26..."
3,"{'log': -9.263805386224244e-07, 'type:': -9.26..."
4,"{'log': -9.263805386224244e-07, 'type:': -9.26..."
...,...
53968,"{'log': -6.617003847303031e-07, 'type:': -6.61..."
53969,"{'log': -8.05548294454282e-07, 'type:': -8.055..."
53970,"{'log': -6.617003847303031e-07, 'type:': -6.61..."
53971,"{'log': -8.05548294454282e-07, 'type:': -8.055..."


In [10]:
# Expand the tfidf dicts to a full matrix
# tfidf_matrix = pd.DataFrame(df["tfidf"].tolist()).fillna(0)


# Isolation Forest From Scratch

**Tree Node Class**

In [11]:
import numpy as np
import pandas as pd
import random
import math

class IsolationTree:
    def __init__(self, depth=0, max_depth=10):
        self.depth = depth
        self.max_depth = max_depth
        self.left = None
        self.right = None
        self.split_attr = None
        self.split_val = None

    def fit(self, X):
        if self.depth >= self.max_depth or len(X) <= 1:
            return

        self.split_attr = random.choice(X.columns)
        col = X[self.split_attr]

        min_val, max_val = col.min(), col.max()
        if min_val == max_val:
            return

        self.split_val = random.uniform(min_val, max_val)

        left_mask = col < self.split_val
        right_mask = col >= self.split_val

        self.left = IsolationTree(self.depth + 1, self.max_depth)
        self.left.fit(X[left_mask])

        self.right = IsolationTree(self.depth + 1, self.max_depth)
        self.right.fit(X[right_mask])

    def path_length(self, x):
        if self.left is None or self.right is None:
            return self.depth
        if x[self.split_attr] < self.split_val:
            return self.left.path_length(x)
        else:
            return self.right.path_length(x)


**Forest + Anomaly Score**

In [12]:
class IsolationForestScratch:
    def __init__(self, n_trees=100, max_depth=10):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X):
        self.X = X.reset_index(drop=True)
        self.n = len(X)
        for _ in range(self.n_trees):
            sample = self.X.sample(frac=1.0, replace=True).reset_index(drop=True)
            tree = IsolationTree(depth=0, max_depth=self.max_depth)
            tree.fit(sample)
            self.trees.append(tree)

    def path_length(self, x):
        return np.mean([tree.path_length(x) for tree in self.trees])

    def c(self, n):
        if n <= 1:
            return 1
        return 2 * (np.log(n - 1) + 0.5772) - 2 * (n - 1) / n

    def anomaly_score(self, x):
        h = self.path_length(x)
        return 2 ** (-h / self.c(self.n))

    def predict(self, X):
        scores = X.apply(lambda x: self.anomaly_score(x), axis=1)
        return scores.apply(lambda s: -1 if s >= 0.6 else 1), scores  # You can tune 0.6


**Apply to df["tfidf"]**

In [ ]:
# Step 1: Convert tfidf dict to matrix
tfidf_matrix = pd.DataFrame(df["tfidf"].tolist()).fillna(0)

# Step 2: Fit our custom Isolation Forest
iso_forest = IsolationForestScratch(n_trees=100, max_depth=10)
iso_forest.fit(tfidf_matrix)

# Step 3: Predict anomalies
df['anomaly'], df['anomaly_score'] = iso_forest.predict(tfidf_matrix)
